In [20]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [21]:
file_path = 'Data/airports.csv'

In [22]:
df = pd.read_csv(file_path)

In [23]:
df.head(5)

,faa,name,lat,lon,alt,tz,dst,tzone
0,AAF,Apalachicola Regional Airport,29.727501,-85.027496,20,-5.0,A,America/New_York
1,AAP,Andrau Airpark,29.722500,-95.588303,79,-6.0,A,America/Chicago
2,ABE,Lehigh Valley International Airport,40.652100,-75.440804,393,-5.0,A,America/New_York
3,ABI,Abilene Regional Airport,32.411301,-99.681900,1791,-6.0,A,America/Chicago
4,ABL,Ambler Airport,67.106300,-157.856989,334,-9.0,A,America/Anchorage


## Exploration Plotly Express scatter_geo

In [32]:
px.scatter_geo(df, lat="lat", lon="lon",  color="alt", projection="natural earth")


##Plot aiports fight routes

In [ ]:
def is_american_faa(faa_code):
    """
    Determines if an FAA code is American based on its timezone. Checks timezone and if it starts with "America/".

    Parameters:
    faa_code (str): The FAA code to check.

    Returns:
    bool: True if the FAA code is American, False otherwise.
    """
    # Filter the DataFrame for the given FAA code
    airport_row = df[df['faa'] == faa_code]

    if airport_row.empty:
        print(f"FAA code {faa_code} not found in the dataset.")
        return False

    # Get the timezone of the airport
    airport_tz = airport_row.iloc[0]['tzone']

    # Check if the timezone starts with "America/"
    return airport_tz.startswith("America/")

The next fuction takes in FAA codes and draws a line from input to NYC

In [44]:
def plot_airport_routes_plotly(faa_codes):
    # Convert a single string to a list, if necessary
    if isinstance(faa_codes, str):
        faa_codes = [faa_codes]
    
    # NYC coordinates
    nyc_lat, nyc_lon = 40.7128, -74.0060
    
    # Filter the DataFrame for the selected airports (assuming your DataFrame is named `df`)
    df_selected = df[df["faa"].isin(faa_codes)]
    
    if df_selected.empty:
        print("No matching FAA codes found.")
        return

    # Check if all airports are in the US (requires a function is_american_faa)
    all_in_us = all(is_american_faa(faa) for faa in faa_codes)
    
    # Choose the projection type based on whether the airports are in the US
    projection_type = "albers usa" if all_in_us else "natural earth"
    
    # Create the base scatter_geo plot for the airports
    fig = px.scatter_geo(
        df_selected,
        lat="lat",
        lon="lon", 
        hover_name="name",
        projection=projection_type,
        title="Routes from NYC to Selected Airport(s)"
    )
    
    # Add NYC point
    fig.add_trace(go.Scattergeo(
        lon=[nyc_lon],
        lat=[nyc_lat],
        mode='markers',
        marker=dict(size=8, color='red'),
        showlegend=False
    ))
    
    # Add lines from NYC to each airport
    for _, row in df_selected.iterrows():
        fig.add_trace(go.Scattergeo(
            lon=[nyc_lon, row['lon']],
            lat=[nyc_lat, row['lat']],
            mode='lines',
            line=dict(width=2, color='green', dash='dash'),
            showlegend=False
        ))
    
    # Update layout: restrict scope to USA if all airports are in the US
    if all_in_us:
        fig.update_geos(
            scope="usa",
            projection_type="albers usa"
        )
    else:
        fig.update_geos(
            scope="world"
        )
    
    fig.show()

In [45]:
# Plot a single airport route
plot_airport_routes_plotly(["AAF", "AAP"])
plot_airport_routes_plotly(["TZR", "AAP", "AAF"])